In [1]:
from utils import *
from model3 import build_decoder_model

%load_ext autoreload
%autoreload 2

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vocab = Vocab("finished_files/vocab", VOCAB_SIZE)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [3]:
model = build_decoder_model(None)

In [4]:
#model = build_decoder_model(None)
model.load_model("full_retr")
torch.cuda.memory_allocated()

271219712

In [6]:
class TestBatcher():
    
    def __init__(self, vocab, filename, max_article_len, max_target_len):
        generator = example_gen(filename)

        self.batches = []
        while True:
            
            try:
                article_text, abstract_text = next(generator)
                
                article_ids, unknown_words_list = article2ids(article_text, vocab)
                art_len = min(max_article_len, len(article_ids))
                article_ids = article_ids[:art_len]
                
                target = abstract2ids(abstract_text, vocab, unknown_words_list)
                tar_len = min(max_target_len, len(target))
                target.append(STOP_DECODING_ID)
                target = target[:tar_len]
                
                article_ids_unk = [i if i < VOCAB_SIZE else UNKNOWN_TOKEN_ID for i in article_ids]
                
                self.batches.append((np.array([article_ids]).T,np.array([article_ids_unk]).T, np.array(target), unknown_words_list))
                #print(article_text)
            except:
                break
        print(len(self.batches))
    
    def generator(self):
        for batch in self.batches:
            yield batch

In [7]:
test_data = TestBatcher(vocab, "finished_files/test.bin", 400, 120)

11490


In [10]:
import codecs
from beam_search import run_beam_search
import os

def test_generate(model, data, beam_width, mod=0):  
    model.train(False)
    sbleu = 0
    with torch.autograd.set_grad_enabled(False):
        for i, (article_text_full, article_text, target, unk_words) in enumerate(data.generator()):
            #result = generate_summary(article_text, model, beam_width)
            
            '''X_batch = LongTensor(article_text)
            inputs = InputLayerState("input", True, X_batch)
            batch_size = X_batch.shape[1]
            symbols = LongTensor([[START_DECODING_ID]*batch_size])
            result = []
            model.eval_run_encoder(inputs, beam_search=False)
            for k in range(100):
                hidden = model.decode(symbols)
                symbols = hidden.argmax(-1)
                result.append(symbols.squeeze(0).item())
            '''
            X_batch, X_batch_full = LongTensor(article_text), LongTensor(article_text_full)
            inputs = [InputLayerState("input", True, X_batch), InputLayerState("input_full", True, X_batch_full)]
            result = run_beam_search(model, vocab, inputs, beam_width, 100)
            #print(result)
            #print(target)
            
            target = clean_sentence(target)
            result = clean_sentence(result)
            bleu = sentence_bleu([result], target, weights=(0.5, 0.5, 0., 0.))
            result = outputids2words(result, vocab, unk_words)
            target = outputids2words(target, vocab, unk_words)
            #print('{:.4f}'.format(bleu))
            sbleu += bleu
            '''if (bleu < 1e-3):
                print(result)
                print(0)
                print(target)'''
            #print(result.split())
            #print('target is \n')
            #print(target.split())
            if not os.access('refs' + str(mod), os.F_OK):
                os.mkdir('refs' + str(mod), 777)
            if not os.access('res' + str(mod), os.F_OK):
                os.mkdir('res' + str(mod), 777)
            with codecs.open('refs' + str(mod) + '/ref.' + str(i) +'.txt', 'w', encoding='utf8') as reference_file:
                sentences = target.split('.')
                for s in sentences:
                    reference_file.write(s.strip() + '.\n')
                
                
            with codecs.open('res' + str(mod) + '/res.' + str(i) + '.txt', 'w', encoding='utf8') as result_file:
                sentences = result.split('.')
                for s in sentences:
                    result_file.write(s.strip() + '.\n')
        print(sbleu/1000)

In [11]:
test_generate(model, test_data, beam_width=4)

c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1.5961139994759272


In [12]:
from pyrouge import Rouge155

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res0'
r.model_dir = 'E:/NIR/drag/refs0'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2019-06-10 12:34:52,343 [MainThread  ] [INFO ]  Writing summaries.
2019-06-10 12:34:52,345 [MainThread  ] [INFO ]  Processing summaries. Saving system files to C:\Users\Anatoly\AppData\Local\Temp\tmpvckcakid\system and model files to C:\Users\Anatoly\AppData\Local\Temp\tmpvckcakid\model.
2019-06-10 12:34:52,346 [MainThread  ] [INFO ]  Processing files in E:/NIR/drag/res0.
2019-06-10 12:34:52,358 [MainThread  ] [INFO ]  Processing res.0.txt.
2019-06-10 12:34:52,366 [MainThread  ] [INFO ]  Processing res.1.txt.
2019-06-10 12:34:52,373 [MainThread  ] [INFO ]  Processing res.10.txt.
2019-06-10 12:34:52,382 [MainThread  ] [INFO ]  Processing res.100.txt.
2019-06-10 12:34:52,390 [MainThread  ] [INFO ]  Processing res.1000.txt.
2019-06-10 12:34:52,397 [MainThread  ] [INFO ]  Processing res.10000.txt.
2019-06-10 12:34:52,402 [MainThread  ] [INFO ]  Processing res.10001.txt.
2019-06-10 12:34:52,408 [MainThread  ] [INFO ]  Processing res.10002.txt.
2019-06-10 12:34:52,412 [MainThread  ] [INFO ] 

2019-06-10 12:34:53,026 [MainThread  ] [INFO ]  Processing res.10092.txt.
2019-06-10 12:34:53,031 [MainThread  ] [INFO ]  Processing res.10093.txt.
2019-06-10 12:34:53,037 [MainThread  ] [INFO ]  Processing res.10094.txt.
2019-06-10 12:34:53,043 [MainThread  ] [INFO ]  Processing res.10095.txt.
2019-06-10 12:34:53,049 [MainThread  ] [INFO ]  Processing res.10096.txt.
2019-06-10 12:34:53,054 [MainThread  ] [INFO ]  Processing res.10097.txt.
2019-06-10 12:34:53,059 [MainThread  ] [INFO ]  Processing res.10098.txt.
2019-06-10 12:34:53,064 [MainThread  ] [INFO ]  Processing res.10099.txt.
2019-06-10 12:34:53,069 [MainThread  ] [INFO ]  Processing res.101.txt.
2019-06-10 12:34:53,080 [MainThread  ] [INFO ]  Processing res.1010.txt.
2019-06-10 12:34:53,087 [MainThread  ] [INFO ]  Processing res.10100.txt.
2019-06-10 12:34:53,096 [MainThread  ] [INFO ]  Processing res.10101.txt.
2019-06-10 12:34:53,102 [MainThread  ] [INFO ]  Processing res.10102.txt.
2019-06-10 12:34:53,107 [MainThread  ] [I

2019-06-10 12:34:53,692 [MainThread  ] [INFO ]  Processing res.10192.txt.
2019-06-10 12:34:53,697 [MainThread  ] [INFO ]  Processing res.10193.txt.
2019-06-10 12:34:53,703 [MainThread  ] [INFO ]  Processing res.10194.txt.
2019-06-10 12:34:53,708 [MainThread  ] [INFO ]  Processing res.10195.txt.
2019-06-10 12:34:53,714 [MainThread  ] [INFO ]  Processing res.10196.txt.
2019-06-10 12:34:53,723 [MainThread  ] [INFO ]  Processing res.10197.txt.
2019-06-10 12:34:53,728 [MainThread  ] [INFO ]  Processing res.10198.txt.
2019-06-10 12:34:53,734 [MainThread  ] [INFO ]  Processing res.10199.txt.
2019-06-10 12:34:53,739 [MainThread  ] [INFO ]  Processing res.102.txt.
2019-06-10 12:34:53,746 [MainThread  ] [INFO ]  Processing res.1020.txt.
2019-06-10 12:34:53,758 [MainThread  ] [INFO ]  Processing res.10200.txt.
2019-06-10 12:34:53,764 [MainThread  ] [INFO ]  Processing res.10201.txt.
2019-06-10 12:34:53,769 [MainThread  ] [INFO ]  Processing res.10202.txt.
2019-06-10 12:34:53,773 [MainThread  ] [I

2019-06-10 12:34:54,333 [MainThread  ] [INFO ]  Processing res.10292.txt.
2019-06-10 12:34:54,338 [MainThread  ] [INFO ]  Processing res.10293.txt.
2019-06-10 12:34:54,343 [MainThread  ] [INFO ]  Processing res.10294.txt.
2019-06-10 12:34:54,349 [MainThread  ] [INFO ]  Processing res.10295.txt.
2019-06-10 12:34:54,354 [MainThread  ] [INFO ]  Processing res.10296.txt.
2019-06-10 12:34:54,359 [MainThread  ] [INFO ]  Processing res.10297.txt.
2019-06-10 12:34:54,364 [MainThread  ] [INFO ]  Processing res.10298.txt.
2019-06-10 12:34:54,370 [MainThread  ] [INFO ]  Processing res.10299.txt.
2019-06-10 12:34:54,375 [MainThread  ] [INFO ]  Processing res.103.txt.
2019-06-10 12:34:54,384 [MainThread  ] [INFO ]  Processing res.1030.txt.
2019-06-10 12:34:54,391 [MainThread  ] [INFO ]  Processing res.10300.txt.
2019-06-10 12:34:54,397 [MainThread  ] [INFO ]  Processing res.10301.txt.
2019-06-10 12:34:54,402 [MainThread  ] [INFO ]  Processing res.10302.txt.
2019-06-10 12:34:54,407 [MainThread  ] [I

2019-06-10 12:34:54,972 [MainThread  ] [INFO ]  Processing res.10392.txt.
2019-06-10 12:34:54,978 [MainThread  ] [INFO ]  Processing res.10393.txt.
2019-06-10 12:34:54,988 [MainThread  ] [INFO ]  Processing res.10394.txt.
2019-06-10 12:34:55,004 [MainThread  ] [INFO ]  Processing res.10395.txt.
2019-06-10 12:34:55,009 [MainThread  ] [INFO ]  Processing res.10396.txt.
2019-06-10 12:34:55,014 [MainThread  ] [INFO ]  Processing res.10397.txt.
2019-06-10 12:34:55,019 [MainThread  ] [INFO ]  Processing res.10398.txt.
2019-06-10 12:34:55,025 [MainThread  ] [INFO ]  Processing res.10399.txt.
2019-06-10 12:34:55,030 [MainThread  ] [INFO ]  Processing res.104.txt.
2019-06-10 12:34:55,037 [MainThread  ] [INFO ]  Processing res.1040.txt.
2019-06-10 12:34:55,046 [MainThread  ] [INFO ]  Processing res.10400.txt.
2019-06-10 12:34:55,051 [MainThread  ] [INFO ]  Processing res.10401.txt.
2019-06-10 12:34:55,063 [MainThread  ] [INFO ]  Processing res.10402.txt.
2019-06-10 12:34:55,069 [MainThread  ] [I

...

2019-06-10 12:37:05,899 [MainThread  ] [INFO ]  Processing ref.9689.txt.
2019-06-10 12:37:05,905 [MainThread  ] [INFO ]  Processing ref.969.txt.
2019-06-10 12:37:05,912 [MainThread  ] [INFO ]  Processing ref.9690.txt.
2019-06-10 12:37:05,917 [MainThread  ] [INFO ]  Processing ref.9691.txt.
2019-06-10 12:37:05,922 [MainThread  ] [INFO ]  Processing ref.9692.txt.
2019-06-10 12:37:05,927 [MainThread  ] [INFO ]  Processing ref.9693.txt.
2019-06-10 12:37:05,933 [MainThread  ] [INFO ]  Processing ref.9694.txt.
2019-06-10 12:37:05,938 [MainThread  ] [INFO ]  Processing ref.9695.txt.
2019-06-10 12:37:05,943 [MainThread  ] [INFO ]  Processing ref.9696.txt.
2019-06-10 12:37:05,949 [MainThread  ] [INFO ]  Processing ref.9697.txt.
2019-06-10 12:37:05,954 [MainThread  ] [INFO ]  Processing ref.9698.txt.
2019-06-10 12:37:05,959 [MainThread  ] [INFO ]  Processing ref.9699.txt.
2019-06-10 12:37:05,964 [MainThread  ] [INFO ]  Processing ref.97.txt.
2019-06-10 12:37:05,971 [MainThread  ] [INFO ]  Proces

2019-06-10 12:37:06,518 [MainThread  ] [INFO ]  Processing ref.9790.txt.
2019-06-10 12:37:06,523 [MainThread  ] [INFO ]  Processing ref.9791.txt.
2019-06-10 12:37:06,529 [MainThread  ] [INFO ]  Processing ref.9792.txt.
2019-06-10 12:37:06,534 [MainThread  ] [INFO ]  Processing ref.9793.txt.
2019-06-10 12:37:06,550 [MainThread  ] [INFO ]  Processing ref.9794.txt.
2019-06-10 12:37:06,555 [MainThread  ] [INFO ]  Processing ref.9795.txt.
2019-06-10 12:37:06,560 [MainThread  ] [INFO ]  Processing ref.9796.txt.
2019-06-10 12:37:06,565 [MainThread  ] [INFO ]  Processing ref.9797.txt.
2019-06-10 12:37:06,571 [MainThread  ] [INFO ]  Processing ref.9798.txt.
2019-06-10 12:37:06,576 [MainThread  ] [INFO ]  Processing ref.9799.txt.
2019-06-10 12:37:06,581 [MainThread  ] [INFO ]  Processing ref.98.txt.
2019-06-10 12:37:06,588 [MainThread  ] [INFO ]  Processing ref.980.txt.
2019-06-10 12:37:06,595 [MainThread  ] [INFO ]  Processing ref.9800.txt.
2019-06-10 12:37:06,600 [MainThread  ] [INFO ]  Proces

2019-06-10 12:37:07,159 [MainThread  ] [INFO ]  Processing ref.9891.txt.
2019-06-10 12:37:07,168 [MainThread  ] [INFO ]  Processing ref.9892.txt.
2019-06-10 12:37:07,173 [MainThread  ] [INFO ]  Processing ref.9893.txt.
2019-06-10 12:37:07,192 [MainThread  ] [INFO ]  Processing ref.9894.txt.
2019-06-10 12:37:07,197 [MainThread  ] [INFO ]  Processing ref.9895.txt.
2019-06-10 12:37:07,202 [MainThread  ] [INFO ]  Processing ref.9896.txt.
2019-06-10 12:37:07,207 [MainThread  ] [INFO ]  Processing ref.9897.txt.
2019-06-10 12:37:07,213 [MainThread  ] [INFO ]  Processing ref.9898.txt.
2019-06-10 12:37:07,219 [MainThread  ] [INFO ]  Processing ref.9899.txt.
2019-06-10 12:37:07,224 [MainThread  ] [INFO ]  Processing ref.99.txt.
2019-06-10 12:37:07,231 [MainThread  ] [INFO ]  Processing ref.990.txt.
2019-06-10 12:37:07,233 [MainThread  ] [INFO ]  Processing ref.9900.txt.
2019-06-10 12:37:07,238 [MainThread  ] [INFO ]  Processing ref.9901.txt.
2019-06-10 12:37:07,243 [MainThread  ] [INFO ]  Proces

2019-06-10 12:37:07,797 [MainThread  ] [INFO ]  Processing ref.9993.txt.
2019-06-10 12:37:07,802 [MainThread  ] [INFO ]  Processing ref.9994.txt.
2019-06-10 12:37:07,807 [MainThread  ] [INFO ]  Processing ref.9995.txt.
2019-06-10 12:37:07,813 [MainThread  ] [INFO ]  Processing ref.9996.txt.
2019-06-10 12:37:07,818 [MainThread  ] [INFO ]  Processing ref.9997.txt.
2019-06-10 12:37:07,823 [MainThread  ] [INFO ]  Processing ref.9998.txt.
2019-06-10 12:37:07,828 [MainThread  ] [INFO ]  Processing ref.9999.txt.
2019-06-10 12:37:07,833 [MainThread  ] [INFO ]  Saved processed files to C:\Users\Anatoly\AppData\Local\Temp\tmpvckcakid\model.
2019-06-10 12:41:18,327 [MainThread  ] [INFO ]  Written ROUGE configuration to C:\Users\Anatoly\AppData\Local\Temp\tmp2wy10u0p\rouge_conf.xml
2019-06-10 12:41:18,327 [MainThread  ] [INFO ]  Running ROUGE with command perl E:\NIR\pyrouge\tools\ROUGE-1.5.5\ROUGE-1.5.5.pl -e E:\NIR\pyrouge\tools\ROUGE-1.5.5\data -c 95 -2 -1 -U -r 1000 -n 4 -w 1.2 -a -m C:\Users\

---------------------------------------------
1 ROUGE-1 Average_R: 0.48046 (95%-conf.int. 0.47802 - 0.48275)
1 ROUGE-1 Average_P: 0.27043 (95%-conf.int. 0.26842 - 0.27238)
1 ROUGE-1 Average_F: 0.33669 (95%-conf.int. 0.33465 - 0.33854)
---------------------------------------------
1 ROUGE-2 Average_R: 0.19299 (95%-conf.int. 0.19048 - 0.19532)
1 ROUGE-2 Average_P: 0.10844 (95%-conf.int. 0.10692 - 0.10989)
1 ROUGE-2 Average_F: 0.13494 (95%-conf.int. 0.13318 - 0.13656)
---------------------------------------------
1 ROUGE-3 Average_R: 0.10670 (95%-conf.int. 0.10456 - 0.10873)
1 ROUGE-3 Average_P: 0.06041 (95%-conf.int. 0.05909 - 0.06165)
1 ROUGE-3 Average_F: 0.07485 (95%-conf.int. 0.07331 - 0.07631)
---------------------------------------------
1 ROUGE-4 Average_R: 0.06925 (95%-conf.int. 0.06738 - 0.07109)
1 ROUGE-4 Average_P: 0.03949 (95%-conf.int. 0.03837 - 0.04060)
1 ROUGE-4 Average_F: 0.04872 (95%-conf.int. 0.04742 - 0.05006)
---------------------------------------------
1 ROUGE-L Aver

model.load_model("cov_loss2")

test_generate(model,test_data, beam_width=4, mod=1)

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res1'
r.model_dir = 'E:/NIR/drag/refs1'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.A.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

In [ ]:
0.10897